In [1]:
%reload_ext autoreload
%autoreload 2

# Imports
Import the required classes and functions.

In [2]:
from mleko.dataset.convert import CSVToVaexConverter
from mleko.dataset.feature_select import MissingRateFeatureSelector, StandardDeviationFeatureSelector, CompositeFeatureSelector
from mleko.dataset.ingest import KaggleIngester
from mleko.dataset.split import ExpressionSplitter, RandomSplitter
from mleko.pipeline import Pipeline
from mleko.pipeline.steps import ConvertStep, FeatureSelectStep, IngestStep, SplitStep

# Constants
Define configuration variables.

In [3]:
OWNER_SLUG = 'mlg-ulb'
DATASET_SLUG = 'creditcardfraud'
DATASET_NAME = f'{OWNER_SLUG}/{DATASET_SLUG}'

TARGET_FEATURE = "Class"
TIME_FEATURE = "Time"
META_FEATURES = [TIME_FEATURE, TARGET_FEATURE]
RANDOM_STATE = 1337

# Pipeline Setup

In [4]:
kaggle_data_source = KaggleIngester(
    output_directory=f"data/{DATASET_NAME}/raw", owner_slug=OWNER_SLUG, dataset_slug=DATASET_SLUG
)
csv_to_arrow_converter = CSVToVaexConverter(
    output_directory=f"data/{DATASET_NAME}/converted", downcast_float=True, random_state=RANDOM_STATE
)
random_data_splitter = RandomSplitter(
    cache_directory=f"data/{DATASET_NAME}/split",
    data_split=(0.80, 0.20),
    shuffle=True,
    stratify=TARGET_FEATURE,
    random_state=RANDOM_STATE,
)
expression_data_splitter = ExpressionSplitter(cache_directory=f"data/{DATASET_NAME}/split", expression="Time > 100")
composite_feature_selector = CompositeFeatureSelector(
    cache_directory=f"data/{DATASET_NAME}/feature_selection",
    feature_selectors=[
        MissingRateFeatureSelector(
            cache_directory=f"data/{DATASET_NAME}/feature_selection",
            missing_rate_threshold=0.7,
            ignore_features=META_FEATURES,
        ),
    ],
)


pipeline = Pipeline(
    steps=[
        IngestStep(kaggle_data_source, outputs=["raw_csv"]),
        ConvertStep(csv_to_arrow_converter, inputs=["raw_csv"], outputs=["df_clean"]),
        SplitStep(random_data_splitter, inputs=["df_clean"], outputs=["df_train_validate", "df_test"]),
        SplitStep(expression_data_splitter, inputs=["df_train_validate"], outputs=["df_train", "df_validate"]),
        FeatureSelectStep(
            composite_feature_selector,
            inputs=["df_train"],
            outputs=["df_train_features_selected"],
        ),
    ]
)


[2023-05-27 22:10:43] [INFO] Attempting to fetch Kaggle API credentials from environment variables 'KAGGLE_USERNAME' and 'KAGGLE_KEY'. (kaggle_ingester.py:74)
[2023-05-27 22:10:43] [WARNING] Kaggle API credentials not found in environment variables, attempting to fetch from fallback path at ~/.kaggle/kaggle.json. (kaggle_ingester.py:82)
[2023-05-27 22:10:43] [INFO] Kaggle credentials successfully fetched. (kaggle_ingester.py:91)


# Run Pipeline

In [5]:
data_container =  pipeline.run().data

[2023-05-27 22:10:46] [INFO] No data container provided. Creating an empty one. (pipeline.py:77)
[2023-05-27 22:10:46] [INFO] Executing step 1: IngestStep. (pipeline.py:81)
[2023-05-27 22:10:46] [INFO] Cache Miss: Downloading mlg-ulb/creditcardfraud/* to data/mlg-ulb/creditcardfraud/raw from Kaggle. (kaggle_ingester.py:292)


[2023-05-27 22:10:51] [INFO] Finished downloading 1 files from Kaggle. (kaggle_ingester.py:303)
[2023-05-27 22:10:51] [INFO] Finished step 1 execution. (pipeline.py:83)
[2023-05-27 22:10:51] [INFO] Executing step 2: ConvertStep. (pipeline.py:81)


[2023-05-27 22:10:51] [INFO] Cache Miss (LRUCache) CSVToVaexConverter.convert: Executing method. (cache_mixin.py:129)


Writing DataFrame to Arrow file: 100%|██████████| 100/100 [00:00<00:00, 780.23it/s]

[2023-05-27 22:10:58] [INFO] Finished step 2 execution. (pipeline.py:83)
[2023-05-27 22:10:58] [INFO] Executing step 3: SplitStep. (pipeline.py:81)
[2023-05-27 22:10:58] [INFO] Cache Miss (LRUCache) RandomSplitter.split: Executing method. (cache_mixin.py:129)
[2023-05-27 22:10:58] [INFO] Shuffling data before splitting. (random_splitter.py:122)
[2023-05-27 22:10:58] [INFO] Splitting data with stratification on column 'Class'. (random_splitter.py:126)


...

[2023-05-27 22:10:59] [INFO] Finished step 4 execution. (pipeline.py:83)
[2023-05-27 22:10:59] [INFO] Executing step 5: FeatureSelectStep. (pipeline.py:81)
[2023-05-27 22:10:59] [INFO] Cache Miss (LRUCache) CompositeFeatureSelector.select_features: Executing method. (cache_mixin.py:129)
[2023-05-27 22:10:59] [INFO] Selecting features from the following set: frozenset({'V7', 'V5', 'V3', 'V27', 'V11', 'V28', 'V25', 'V15', 'V4', 'V12', 'V10', 'V19', 'V21', 'V8', 'V23', 'V2', 'V26', 'V1', 'V22', 'V9', 'V14', 'V18', 'V24', 'V16', 'V17', 'Amount', 'V20', 'V13', 'V6'}). (missing_rate_feature_selector.py:100)



Calculating missing rates for features: 100%|██████████| 29/29 [00:00<00:00, 57.78it/s]

[2023-05-27 22:10:59] [INFO] Dropping features with missing rate >= 0.7: set(). (missing_rate_feature_selector.py:110)



Writing DataFrame to Arrow file: 100%|██████████| 100/100 [00:00<00:00, 957.18it/s]

[2023-05-27 22:11:00] [INFO] Finished step 5 execution. (pipeline.py:83)
